In [1]:
#https://chromedriver.chromium.org/downloads - download based on your version of chrome - chrome://settings/help
#you will need to install lxml - pip install lxml
# https://www.psycopg.org/docs/ - pip install psycopg
import psycopg2
import datetime
import time as ti
import sys
import requests
import itertools
from requests.exceptions import ConnectionError
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import html5lib
from selenium import webdriver
from selenium.common.exceptions import NoAlertPresentException
stock_code = 'NVDA'
driver = webdriver.Chrome()
url = 'https://finance.yahoo.com/quote/' + stock_code + '?p=' + stock_code + '&.tsrc=fin-srch'
r = requests.get(url)

conn = psycopg2.connect(host="localhost", database="TradingAlgo", user="postgres", password="Sheeeeeeeeesh", port = '5432')
cursor = conn.cursor()
cursor.execute("select version()")
data = cursor.fetchone()
print("Connection established to: ",data)
print("Checking for existing ticker: ",stock_code)
cursor.execute("SELECT * FROM Stock WHERE Stock.Ticker = '%s'" % stock_code)
Dcheck = cursor.fetchone()
if Dcheck != None:
        print(Dcheck, " Exists in the database")

counter = 0
while True:
    stock_code = 'NVDA'
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36",
    "Cache-Control": "no-cache, max-age=0",  # disable caching
}
    
    counter += 1
    A = 'My(6px) Pos(r) smartphone_Mt(6px) W(100%)'
    B = 'D(ib) Mend(20px)' # Price
    C = 'Fw(b) Fz(36px) Mb(-4px) D(ib)'
    D = 'W(100%)'
    E = 'Fz(xs) Mb(4px)'
    F = 'W(100%) M(0) Bdcl(c)'
    # Left Table
    G = 'D(ib) W(1/2) Bxz(bb) Pend(12px) Va(t) ie-7_D(i) smartphone_D(b) smartphone_W(100%) smartphone_Pend(0px) smartphone_BdY smartphone_Bdc($seperatorColor)'
    #tr class
    H = 'Bxz(bb) Bdbw(1px) Bdbs(s) Bdc($seperatorColor) H(36px) '
        
    def Connection_Error():
        display('Something went wrong :(')
    
    def web_content_div(web_content, class_path):
        
        web_content_div = web_content.find_all('div', {'class': class_path})
        
        try:
            spans = web_content_div[0].find_all('span')
            textA = [span.get_text() for span in spans]
        except IndexError:
            textA = []
            
        return textA
    
    def web_content_td(web_content, class_path):
        
        web_content_td = web_content.find_all('td', {'class': class_path})
        
        
        
        try:
            spans = web_content_td[0].find_all('span')
            textB = [span.get_text() for span in spans]
        except IndexError:
            textB = []
            
        return textB
    
    def price(stock_code):
            
        cursor.execute("SELECT * FROM Stock WHERE Stock.Ticker = '%s'" % stock_code)
        Dcheck = cursor.fetchone()
        Dmerge = pd.DataFrame(Dcheck, columns = ['Statistics'])
        r = requests.get(url)
        web_content = BeautifulSoup(requests.get(url, headers=headers).text, 'lxml')
        
        #tbl = web_content.find('table', {'class': 'W(100%)'})
        #tblB = web_content.find('table', {'class': 'W(100%) M(0) Bdcl(c)'})
        #tbl.append(tblB)
        #rows = tbl.tbody.find_all('tr')
        #rowsB = tblB.tbody.find_all('tr')
        #rows.append(rowsB)
        #vals = rows.find('td')
        #for td in rows[:0].find_all("td"):
            #values.append(td.get_text())
            
        #rows = data.tbody.find_all('tr')
        #d = []
        #for x in rows:
            #d = x.get_text()
        #vals = rows.find_all('td')
        #d = []
        #for val in vals:
            #d = val.get_text()
        #dfs = pd.read_html(url)
        #Tbl = dfs[0]
        
        a = []
        for x in web_content.find('table', {'class': 'W(100%)'}):
            for tr in x:
                for td in tr:
                    try:
                        a.append(td.get_text())
                    except:
                        continue
        b = []
        for x in web_content.find('table', {'class': 'W(100%) M(0) Bdcl(c)'}):
            for tr in x:
                for td in tr:
                    try:
                        b.append(td.get_text())
                    except:
                        continue
        c = []
        CurrentPrice = web_content.find('div', {'class': 'D(ib) Mend(20px)'})
        for fin in CurrentPrice.find('fin-streamer', {'class': 'Fw(b) Fz(36px) Mb(-4px) D(ib)'}):
                    try:
                        c.append(fin.get_text())
                    except:
                        continue

        Price_Change = web_content_div(web_content, B)
        dfa = pd.DataFrame(a, columns = ['Statistics'])
        dfb = pd.DataFrame(b, columns = ['Statistics'])
        dfc = pd.DataFrame(c, columns = ['Price_Change'])
        Price_Change = pd.DataFrame(Price_Change, columns = ['Price_Change'])
        frames = [dfa, dfb]
        stats = pd.concat(frames).reset_index(drop=True)
        PrVals = [dfc, Price_Change]
        PrCh = pd.concat(PrVals).reset_index(drop=True)
        SData = stats.loc[[1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31],:].reset_index(drop=True)
        Sprep = [SData, Dmerge]
        Stbl = pd.concat(Sprep).reset_index(drop=True)[:-1]
        Stbl = Stbl['Statistics']
        
        if c == []:
            print('Could not fetch data from yahoo!')
            exit()
                
        if Dcheck == None:
            #conn = psycopg2.connect(host="localhost", database="TradingAlgo", user="postgres", password="Sheeeeesh", port = '5432')
            #cursor = conn.cursor()
            cursor.execute("INSERT INTO Stock(Ticker) VALUES('%s')" % stock_code)
            cursor.execute("SELECT * FROM Stock WHERE Stock.Ticker = '%s'" % stock_code)
            Dcheck = cursor.fetchone()
            conn.commit()
            
        if counter <= 1:
            cursor.execute("INSERT INTO Statistics(previous_close, open, bid, ask, days_range, years_range, volume, avg_volume, market_cap, beta_5y_monthly, pe_ratio_ttm, eps_ttm, earnings_date, forward_div_yield, ex_div, year_target, stockid) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", Stbl)
            conn.commit()
            display(PrCh), display(stats) 
        else:
            return PrCh
        
    try:    
        price(stock_code)
        for i in range(5,-1,-1):
            print(f"{i} seconds until refresh", end="\r", flush=True)
            ti.sleep(1)
            
    except KeyboardInterrupt:
        print('Process haulted: Database connection closed.')
        conn.close()
  
    except ConnectionError:
        Connection_Error()


OperationalError: connection to server at "localhost" (::1), port 5432 failed: FATAL:  password authentication failed for user "postgres"


In [1]:
pip install jupyterthemes

     ---------------------------------------- 7.0/7.0 MB 17.3 MB/s eta 0:00:00
     ---------------------------------------- 46.7/46.7 KB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 49.6/49.6 KB 2.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
!jt -l

Available Themes: 
   chesterish
   grade3
   gruvboxd
   gruvboxl
   monokai
   oceans16
   onedork
   solarizedd
   solarizedl


In [8]:
!jt -t chesterish

In [5]:
Available Themes: 
   chesterish
   grade3
   gruvboxd
   gruvboxl
   monokai
   oceans16
   onedork
   solarizedd
   solarizedl

SyntaxError: invalid syntax (1615938206.py, line 1)

In [7]:
!jt -r

Reset css and font defaults in:
C:\Users\graso\.jupyter\custom &
C:\Users\graso\AppData\Roaming\jupyter\nbextensions
